# Functioning example for the ZIPRA library
This library provides tools to extract and process the .SAFE file containing Sentinel-2 data

In [5]:
import pandas as pd
from shapely import geometry
import requests
import leafmap as leafmap
from shapely.geometry import shape
import json

## How to get the data

For using the library, a fundamental step is to download the zip folder containing the data. 
Below is a script that guides the user to obtain such data using the Odata endpoint provided by Copernicus Data Space Catalogue [Odata info](https://documentation.dataspace.copernicus.eu/APIs/OData.html)

First this is defining an AOI, use the draw control instruments to draw the desired AOI

In [ ]:
# Draw the AOI on the map below
m = leafmap.Map(center=[45.64, 9.60], zoom=5, toolbar_control=False, fullscreen_control=False)
m

In [7]:
# This section extract the AOI from the map and extracts its bounding box
AOI = m.draw_features
if not AOI:
    print("Please, select an area of interest on the map")
else:
    AOI = shape(AOI[0]['geometry'])
    bbox = AOI.bounds
    print(f"The Shapely geometry: {AOI}")
    print(f"The bounding box in EPSG:4326 is (min_x, min_y, max_x, max_y): {bbox}")

The Shapely geometry: POLYGON ((8.188477 44.548735, 8.188477 46.002059, 12.143555 46.002059, 12.143555 44.548735, 8.188477 44.548735))
The bounding box in EPSG:4326 is (min_x, min_y, max_x, max_y): (8.188477, 44.548735, 12.143555, 46.002059)


In [4]:
# This section allows to define some parameters for searching Sentinel-2 products, 
# if the user already has the ID of the product, they can skip this and the following cell

# Copernicus Data Space Catalogue OData endpoint
catalogue_odata_url = "https://catalogue.dataspace.copernicus.eu/odata/v1"
collection_name = "SENTINEL-2"
product_type = "S2MSI2A"

# USER DEFINED PARAMETERS
# Define the maximum cloud coverage
max_cloud_cover = 30

#Define the period
search_period_start = "2025-09-15T00:00:00.000Z"
search_period_end = "2025-09-30T00:00:00.000Z"
# END OF USER DEFINED PARAMETERS

aoi = geometry.box(*bbox).wkt

In [5]:
search_query = f"{catalogue_odata_url}/Products?$filter=Collection/Name eq '{collection_name}' and Attributes/OData.CSC.StringAttribute/any(att:att/Name eq 'productType' and att/OData.CSC.StringAttribute/Value eq '{product_type}') and OData.CSC.Intersects(area=geography'SRID=4326;{aoi}') and ContentDate/Start gt {search_period_start} and ContentDate/Start lt {search_period_end} and Attributes/OData.CSC.DoubleAttribute/any(att:att/Name eq 'cloudCover' and att/OData.CSC.DoubleAttribute/Value le {max_cloud_cover})"
response = requests.get(search_query).json()
result = pd.DataFrame.from_dict(response["value"])
result.head(5)

,@odata.mediaContentType,Id,Name,ContentType,ContentLength,OriginDate,PublicationDate,ModificationDate,Online,EvictionDate,S3Path,Checksum,ContentDate,Footprint,GeoFootprint
0,application/octet-stream,9bb683fb-7ad2-47e4-804c-5c91c0853192,S2C_MSIL2A_20250929T100821_N0511_R022_T32TNR_2...,application/octet-stream,598037871,2025-09-29T16:46:47.000000Z,2025-09-29T16:51:57.542905Z,2025-09-29T16:53:24.082903Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2025/09/29/S2C_MSIL...,"[{'Value': '422c042439f3100db80c496a33645e61',...","{'Start': '2025-09-29T10:08:21.025000Z', 'End'...",geography'SRID=4326;POLYGON ((9.62737689938526...,"{'type': 'Polygon', 'coordinates': [[[9.627376..."
1,application/octet-stream,1630244b-c7e2-4931-aca1-1b6d3288b1a3,S2C_MSIL2A_20250929T100821_N0511_R022_T32TNQ_2...,application/octet-stream,862256766,2025-09-29T16:46:41.000000Z,2025-09-29T16:53:16.683536Z,2025-09-29T16:54:25.951688Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2025/09/29/S2C_MSIL...,"[{'Value': '8e05435015047f35212a6f62a81a4567',...","{'Start': '2025-09-29T10:08:21.025000Z', 'End'...",geography'SRID=4326;POLYGON ((9.32606909624059...,"{'type': 'Polygon', 'coordinates': [[[9.326069..."
2,application/octet-stream,b3807efb-ba3f-44e2-8c59-c78b8611ffa8,S2C_MSIL2A_20250929T100821_N0511_R022_T32TPQ_2...,application/octet-stream,1217692208,2025-09-29T16:46:57.000000Z,2025-09-29T16:52:37.910951Z,2025-09-29T16:53:27.098917Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2025/09/29/S2C_MSIL...,"[{'Value': 'aea9b1c4fee3f6d5b1646bd7495ba614',...","{'Start': '2025-09-29T10:08:21.025000Z', 'End'...",geography'SRID=4326;POLYGON ((10.2720409677684...,"{'type': 'Polygon', 'coordinates': [[[10.27204..."
3,application/octet-stream,0862ef84-fcbd-418b-abd9-95db4defcb8f,S2A_MSIL2A_20250921T100731_N0511_R022_T32TPR_2...,application/octet-stream,1206252548,2025-09-21T13:10:59.000000Z,2025-09-21T13:19:06.941472Z,2025-09-21T13:20:12.253305Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2025/09/21/S2A_MSIL...,"[{'Value': 'bcaba0b24bb85d5146265e21a9071d2b',...","{'Start': '2025-09-21T10:07:31.024000Z', 'End'...",geography'SRID=4326;POLYGON ((10.2925320358333...,"{'type': 'Polygon', 'coordinates': [[[10.29253..."
4,application/octet-stream,75e67f5d-381e-4fdd-b767-fb53bcfaeec1,S2B_MSIL2A_20250920T102619_N0511_R108_T32TMQ_2...,application/octet-stream,588471893,2025-09-20T15:02:44.000000Z,2025-09-20T15:07:09.961644Z,2025-09-20T15:08:12.583068Z,True,9999-12-31T23:59:59.999999Z,/eodata/Sentinel-2/MSI/L2A/2025/09/20/S2B_MSIL...,"[{'Value': 'e74d79645597fca5f8bbc2b416599a14',...","{'Start': '2025-09-20T10:26:19.024000Z', 'End'...",geography'SRID=4326;POLYGON ((8.52564060898841...,"{'type': 'Polygon', 'coordinates': [[[8.525640..."


In [6]:
# Select the image you want to download

# USER DEFINED PARAMETERS
Index_image = 0  # Change this index to select different images from the search result
path=f"./DATA/"
# END OF USER DEFINED PARAMETERS

Image_Id = result.iloc[Index_image]["Id"]
Image_name = result.iloc[Index_image]["Name"]
url = f"https://download.dataspace.copernicus.eu/odata/v1/Products({Image_Id})/$value"
path = path + f"{Image_name}.zip"

Before proceding with the download, the user should create an account on Copernicus Data Space and get OAuth client, a guide can be found at this [link](https://documentation.dataspace.copernicus.eu/APIs/SentinelHub/Overview/Authentication.html)

In [6]:
# Authentication credentials
# USER DEFINED PARAMETERS
with open("My_credentials.json") as f:
    credentials = json.load(f)

username = credentials["username"]
password = credentials["password"]
# END OF USER DEFINED PARAMETERS

In [7]:
# Get an access token and establish a session
token_url = "https://identity.dataspace.copernicus.eu/auth/realms/CDSE/protocol/openid-connect/token"
token_data = {
    "grant_type": "client_credentials",
    "client_id": username,
    "client_secret": password
}
token_response = requests.post(token_url, data=token_data)
access_token = token_response.json().get("access_token")

print("Token response:", token_response.status_code)
session = requests.Session()
session.headers["Authorization"] = f"Bearer {access_token}"

Token response: 200


In [9]:
#Download the image, this might take few minutes
try:
    response = session.get(url, stream=True)
    response.raise_for_status()
    zip_data = response.content
    response.close()

    # Writes the downloaded content to a file
    with open(path, 'wb') as f:
        f.write(zip_data)

    print(f"\nData saved to '{path}' (Size: {len(zip_data)} bytes).")

except requests.exceptions.RequestException as e:
    print(f"\nAn error occurred during the request: {e}")


Data saved to './DATA/S2C_MSIL2A_20250929T100821_N0511_R022_T32TNR_20250929T155516.SAFE.zip' (Size: 598037871 bytes).


## Zipra library example

In [10]:
from ZIPRA import Band_estraction
tiff_file, band_list = Band_estraction(path)

File ZIP decompressed successfully.
band folder: ./DATA/S2C_MSIL2A_20250929T100821_N0511_R022_T32TNR_20250929T155516.SAFE\GRANULE\L2A_T32TNR_A005569_20250929T101531\IMG_DATA
Band B02 found at ./DATA/S2C_MSIL2A_20250929T100821_N0511_R022_T32TNR_20250929T155516.SAFE\GRANULE\L2A_T32TNR_A005569_20250929T101531\IMG_DATA\R10m\T32TNR_20250929T100821_B02_10m.jp2
Band B03 found at ./DATA/S2C_MSIL2A_20250929T100821_N0511_R022_T32TNR_20250929T155516.SAFE\GRANULE\L2A_T32TNR_A005569_20250929T101531\IMG_DATA\R10m\T32TNR_20250929T100821_B03_10m.jp2
Band B04 found at ./DATA/S2C_MSIL2A_20250929T100821_N0511_R022_T32TNR_20250929T155516.SAFE\GRANULE\L2A_T32TNR_A005569_20250929T101531\IMG_DATA\R10m\T32TNR_20250929T100821_B04_10m.jp2
Band B08 found at ./DATA/S2C_MSIL2A_20250929T100821_N0511_R022_T32TNR_20250929T155516.SAFE\GRANULE\L2A_T32TNR_A005569_20250929T101531\IMG_DATA\R10m\T32TNR_20250929T100821_B08_10m.jp2
Band B12 found at ./DATA/S2C_MSIL2A_20250929T100821_N0511_R022_T32TNR_20250929T155516.SAFE\GRA

In [11]:
print("Band_list:", band_list)
print("Tiff_file:", tiff_file)

Band_list: ['B02', 'B03', 'B04', 'B08', 'B12', 'SCL']
Tiff_file: ./DATA\L2A_T32TNR_A005569_20250929T101531.tif


Area calculations:

In [ ]:
from ZIPRA import Area_calculation
import rasterio

tiff_file="./DATA/L2A_T32TNR_A005569_20250929T101531.tif"
band_list=['B02', 'B03', 'B04', 'B08', 'B12', 'SCL'] #to be deleted when running all code above
class_list = [1,2,3] #example
SCL_band = band_list.index('SCL') + 1  # Get the band number for SCL
Area = Area_calculation(tiff_file, class_list, SCL_band)
print("Total area of specified classes (in m^2):", Area)

Total area of specified classes (in m^2): 232461900.0


In [ ]:
m.set_center(((bbox[0] + bbox[2]) / 2),((bbox[1] + bbox[3]) / 2), zoom=8)
m.add_raster(tiff_file, bands=[band_list.index('B04') + 1, band_list.index('B03') + 1, band_list.index('B02') + 1], layer_name="Sentinel-2 RGB")
m.add_layer_control()
m

In [10]:
# This section extract the AOI clipped from the map and extracts its bounding box
AOI_clipped = m.draw_features[-1] #last drawn feature
if not AOI_clipped:
    print("Please, select an area of interest on the map")
else:
    AOI_clipped = shape(AOI_clipped['geometry'])
    bbox_clipped = AOI_clipped.bounds
    print(f"The Shapely geometry clipped: {AOI_clipped}")
    print(f"The bounding box in EPSG:4326 is (min_x, min_y, max_x, max_y): {bbox_clipped}")

The Shapely geometry clipped: POLYGON ((8.188477 44.548735, 8.188477 46.002059, 12.143555 46.002059, 12.143555 44.548735, 8.188477 44.548735))
The bounding box in EPSG:4326 is (min_x, min_y, max_x, max_y): (8.188477, 44.548735, 12.143555, 46.002059)


In [ ]:
from rasterio import mask
from shapely import wkt
import os
from rasterio.warp import transform_geom
import geopandas as gpd
from shapely.geometry import box

from ZIPRA import Clip_AOI

clipped_tiff = Clip_AOI(tiff_file, AOI_clipped.wkt)
print("Clipped raster data path:", clipped_tiff)

Intersection AOI/raster: [ True]
Clipped raster data path: ./DATA/L2A_T32TNR_A005569_20250929T101531_CLIPPED.tif


In [ ]:
m.add_raster(clipped_tiff, layer_name="Raster clipped")
m